<a href="https://colab.research.google.com/github/Aparnamol-KS/Content-Generation-LLM-project/blob/main/Content_Generation.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

#Install Required Packages

In [6]:
!pip install -q langchain groq sentence-transformers faiss-cpu gradio tiktoken groq langchain-community python-pptx


#Import Required Packages

In [18]:
from langchain_community.embeddings import HuggingFaceEmbeddings
from langchain.document_loaders import TextLoader
from langchain.chains import RetrievalQA
from langchain.prompts import PromptTemplate
from langchain.chains import LLMChain
from pptx import Presentation
from pptx.util import Inches, Pt
from pptx.enum.text import PP_ALIGN
from pptx.dml.color import RGBColor
from pptx.enum.shapes import MSO_SHAPE, MSO_CONNECTOR_TYPE
from sentence_transformers import SentenceTransformer
from langchain.llms.base import LLM
from typing import List, Optional
from groq import Groq
import pandas as pd
import faiss
import numpy as np
import gradio as gr
import random
import tempfile
import re
import os


#Set Up API Keys


In [33]:

from google.colab import userdata
api_key = userdata.get('API_KEY')


#Load LLMs and Embedding Models

In [34]:
class GroqLLM(LLM):
    model: str = "llama3-8b-8192"
    api_key: str = api_key
    temperature: float = 0.0

    def _call(self, prompt: str, stop: Optional[List[str]] = None) -> str:
        client = Groq(api_key=self.api_key)

        messages = [
            {"role": "system", "content": "You are a helpful assistant."},
            {"role": "user", "content": prompt}
        ]

        response = client.chat.completions.create(
            model=self.model,
            messages=messages,
            temperature=self.temperature,
        )

        return response.choices[0].message.content

    @property
    def _llm_type(self) -> str:
        return "groq-llm"


In [35]:
llm = GroqLLM(model="llama3-8b-8192", api_key=api_key, temperature=0.7)

# Define Prompt Templates & Chains

In [29]:
# Prompt templates
name_prompt = PromptTemplate(
    input_variables=["domain"],
    template="Generate 10 creative and catchy startup product name suggestions for the domain: {domain}."
)

description_prompt = PromptTemplate(
    input_variables=["product_name"],
    template="Write a visionary and fictional product description for a startup named '{product_name}'. Make it imaginative and futuristic."
)

image_prompt = PromptTemplate(
    input_variables=["product_description"],
    template="Based on this product description, create a detailed prompt to generate an AI image of the product: {product_description}"
)

slides_prompt = PromptTemplate(
    input_variables=["product_name", "product_description"],
    template="""
You are tasked with creating a professional 10-slide pitch deck for a futuristic and innovative product called '{product_name}'.

Context:
{product_description}

📋 **Instructions**:
- Each slide should include:
  - A **title** only once (you'll generate it separately for each slide).
  - 3–5 **detailed bullet points** with rich, medium-to-long explanations.
- ❌ Do not include headings or slide titles inside the bullet points.
- ❌ Do not include placeholders like [Image:] or [Title:].
- ✅ Only provide well-formed bullet points that can go directly into a PowerPoint presentation.
- ✅ Focus on clear, engaging, and professional startup pitch-style language.

🎯 **Slides**:
Slide 1: Introduction
Slide 2: Product Overview
Slide 3: The Problem
Slide 4: The Solution
Slide 5: Key Features
Slide 6: Market Analysis
Slide 7: Target Audience
Slide 8: Monetization Strategy
Slide 9: Technology Stack
Slide 10: Conclusion

Make the presentation sound like it belongs in a startup pitch competition or innovation expo.
"""
)
# Create chains
name_chain = LLMChain(llm=llm, prompt=name_prompt)
description_chain = LLMChain(llm=llm, prompt=description_prompt)
image_chain = LLMChain(llm=llm, prompt=image_prompt)
slides_chain = LLMChain(llm=llm, prompt=slides_prompt)


#Processing Inputs

In [23]:
def get_selected_name(index, suggestions):
    try:
        index = int(index) - 1  # Adjust for 0-based indexing
        if 0 <= index < len(suggestions):
            name_line = suggestions[index]
            name = name_line.split("**")[1] if "**" in name_line else name_line
            return name
        else:
            return "Invalid index. Please choose a number between 1 and 10."
    except:
        return "Invalid input. Please enter a valid number."

def format_suggestions(raw_lines):
    suggestions = []
    for line in raw_lines:
        if "**" in line and ":" in line:
            # Extract the name and description
            parts = line.split("**")
            if len(parts) >= 3:
                name = parts[1]
                description = parts[2].strip(": ").strip()
                suggestions.append(f"{name}: {description}")
    # Format with numbers
    numbered = [f"{i+1}. {text}" for i, text in enumerate(suggestions)]
    return "\n".join(numbered), suggestions

def get_name_suggestions(domain):
    raw_output = name_chain.run(domain)
    lines = raw_output.split("\n")
    display_text, clean_suggestions = format_suggestions(lines)

    return display_text, clean_suggestions

def get_details_from_index(index, suggestions):
    try:
        idx = int(index) - 1
        product_name = suggestions[idx]
    except (ValueError, IndexError):
        product_name = suggestions[0]

    description = description_chain.run(product_name)
    image_prompt = image_chain.run(description)
    return description, image_prompt

#Slide Generation Logic

In [24]:
def extract_product_name(text):
    # Match word inside optional asterisks or just the first alphanumeric word
    match = re.search(r"\*{0,2}([\w]+)\*{0,2}", text)
    return match.group(1) if match else text.strip()


def clean_text(text):
    return text.replace("*", "").strip()
def generate_ppt_enhanced(product_name, product_description):
    product_name = extract_product_name(product_name)
    slides_text = slides_chain.run(product_name=product_name, product_description=product_description)

    prs = Presentation()

    # 1. Title slide (Intro)
    title_slide_layout = prs.slide_layouts[0]  # Usually title slide layout
    slide = prs.slides.add_slide(title_slide_layout)
    slide.shapes.title.text = product_name
    subtitle = slide.placeholders[1]
    subtitle.text = "Innovative Product Presentation"
    subtitle.text_frame.paragraphs[0].font.italic = True
    subtitle.text_frame.paragraphs[0].font.size = Pt(24)
    subtitle.text_frame.paragraphs[0].font.color.rgb = RGBColor(0, 102, 204)  # blue

    # 2. Content slides from generated text
    for slide_block in slides_text.strip().split("Slide")[1:]:
        lines = slide_block.strip().split('\n')
        title = lines[0].split(':')[1].strip()
        # content = [line.strip("-• ") for line in lines[1:] if line.strip()]
        content = [clean_text(line) for line in lines[1:] if line.strip()]
        # Use Title and Content layout (1)
        slide = prs.slides.add_slide(prs.slide_layouts[1])
        slide.shapes.title.text = title

        text_frame = slide.shapes.placeholders[1].text_frame
        text_frame.clear()  # Clear default content

        for point in content:
            p = text_frame.add_paragraph()
            p.text = point
            p.level = 0
            p.font.size = Pt(18)
            p.font.color.rgb = RGBColor(0, 0, 0)
            p.space_after = Pt(6)


    # 4. Conclusion slide
    concl_slide = prs.slides.add_slide(prs.slide_layouts[1])
    concl_slide.shapes.title.text = "Conclusion"
    concl_tf = concl_slide.shapes.placeholders[1].text_frame
    concl_tf.clear()
    p = concl_tf.add_paragraph()
    p.text = f"{product_name} is poised to transform the industry with its innovative approach and customer-centric design."
    p.font.size = Pt(18)

    # 5. Thank You slide
    thank_slide = prs.slides.add_slide(prs.slide_layouts[6])  # Blank slide
    txBox = thank_slide.shapes.add_textbox(Inches(2), Inches(3), Inches(6), Inches(1))
    tf = txBox.text_frame
    p = tf.add_paragraph()
    p.text = "Thank You!"
    p.font.size = Pt(40)
    p.font.bold = True
    p.alignment = PP_ALIGN.CENTER

    # Save presentation
    tmp_file = tempfile.NamedTemporaryFile(delete=False, suffix=".pptx")
    prs.save(tmp_file.name)

    return tmp_file.name

# Define Vectorstore for Similarity Search

In [25]:
# ---- Model and Stores ----
model = SentenceTransformer("all-MiniLM-L6-v2")
vector_size = 384
product_index = faiss.IndexFlatL2(vector_size)
product_metadata_store = []

# ---- Add Function ----
def add_product(name, description, image_prompt):
    embedding = model.encode([description])[0]
    product_index.add(np.array([embedding]))
    product_metadata_store.append({
        "product_name": name,
        "product_desc": description,
        "image_prompt": image_prompt
    })
    return f"✅ Saved: {name}"

# ---- Search Function ----
def search_similar_products(query):
    if not product_metadata_store:
        return pd.DataFrame(columns=["Product Name", "Description", "Image Prompt"])
    query_embedding = model.encode([query])
    D, I = product_index.search(np.array(query_embedding), k=min(5, len(product_metadata_store)))
    results = []
    for idx in I[0]:
        if idx < len(product_metadata_store):
            item = product_metadata_store[idx]
            results.append([
                item.get("product_name", ""),
                item.get("product_desc", ""),
                item.get("image_prompt", "")
            ])
    return pd.DataFrame(results, columns=["Product Name", "Description", "Image Prompt"])


/usr/local/lib/python3.11/dist-packages/huggingface_hub/utils/_auth.py:94: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


modules.json:   0%|          | 0.00/349 [00:00<?, ?B/s]

config_sentence_transformers.json:   0%|          | 0.00/116 [00:00<?, ?B/s]

README.md:   0%|          | 0.00/10.5k [00:00<?, ?B/s]

sentence_bert_config.json:   0%|          | 0.00/53.0 [00:00<?, ?B/s]

...

tokenizer_config.json:   0%|          | 0.00/350 [00:00<?, ?B/s]

vocab.txt:   0%|          | 0.00/232k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/466k [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/112 [00:00<?, ?B/s]

config.json:   0%|          | 0.00/190 [00:00<?, ?B/s]

#Define Gradio Interface

In [31]:
def surprise_domain():
    return random.choice(["Health", "Education", "Finance", "Travel", "Gaming","Edu-Tech","Smart-home","AI & Tech","Health & Wellness","E-commerce & Marketing"," Education & Learning"])

with gr.Blocks(css="""
body {
    font-family: 'Segoe UI', sans-serif;
}

.frosted-glass {
    background: rgba(255, 255, 255, 0.05);
    border: 1px solid rgba(255, 255, 255, 0.1);
    border-radius: 20px;
    backdrop-filter: blur(12px);
    padding: 24px;
    box-shadow: 0 0 20px rgba(0, 140, 255, 0.3);
    margin: 10px 0;
}

button {
    background: linear-gradient(135deg, #00b4d8, #0077cc) !important;
    color: white !important;
    border: none !important;
    border-radius: 12px !important;
    font-weight: 600;
    padding: 10px 20px !important;
}

button:hover {
    background: linear-gradient(135deg, #0096c7, #005fa3) !important;
}
""") as demo:

    suggested_names_state = gr.State()

    gr.Markdown("<h1 style='text-align:center; color:white;'>🚀 AI Startup Idea Generator</h1>")

    with gr.Tabs():

        with gr.Tab("🚀 Startup Idea Generator"):
            with gr.Column(elem_classes="frosted-glass"):

                with gr.Row():
                    domain_input = gr.Textbox(
                        label="Enter a domain",
                        placeholder="e.g. Education, Finance",
                        scale=3,
                        lines=1,
                        max_lines=1
                    )
                    surprise_btn = gr.Button("🎲 Surprise Me", scale=1)

                gen_names_btn = gr.Button("🔍 Get Product Name Ideas")

                with gr.Row():
                    with gr.Column(elem_classes="frosted-glass"):

                        suggested_names_box = gr.Textbox(
                            label="💡 Product Name Suggestions",
                            lines=12
                        )
                        suggested_names_state = gr.State()

                        idea_index_input = gr.Number(
                            label="Idea #",
                            value=1,
                            precision=0,
                            interactive=True
                        )

                        selected_name_output = gr.Textbox(
                            label="Selected Product Name",
                            interactive=False
                        )

                        gen_details_btn = gr.Button("✨ Generate Description & Image Prompt")

                        desc_output = gr.Textbox(
                            label="📄 Product Description",
                            lines=8,
                            max_lines=10,
                            interactive=False
                        )

                        img_prompt_output = gr.Textbox(
                            label="🖼️ Image Prompt",
                            lines=8,
                            max_lines=6,
                            interactive=False
                        )

                        add_btn = gr.Button("➕ Add Product")
                        status = gr.Textbox(label="Status")

                        generate_ppt_btn = gr.Button("📊 Generate Pitch Deck")

                        pptx_download = gr.File(
                            label="📥 Download Pitch Deck",
                            interactive=False
                        )

                        # Button click handlers
                        add_btn.click(
                            fn=add_product,
                            inputs=[selected_name_output, desc_output, img_prompt_output],
                            outputs=status
                        )

                        gen_names_btn.click(
                            fn=get_name_suggestions,
                            inputs=domain_input,
                            outputs=[suggested_names_box, suggested_names_state]
                        )

                        gen_details_btn.click(
                            fn=get_selected_name,
                            inputs=[idea_index_input, suggested_names_state],
                            outputs=selected_name_output
                        )

                        gen_details_btn.click(
                            fn=get_details_from_index,
                            inputs=[idea_index_input, suggested_names_state],
                            outputs=[desc_output, img_prompt_output]
                        )

                        generate_ppt_btn.click(
                            fn=generate_ppt_enhanced,
                            inputs=[selected_name_output, desc_output],
                            outputs=pptx_download
                        )

                        surprise_btn.click(
                            fn=surprise_domain,
                            outputs=domain_input
                        )

        with gr.Tab("🔍 Search Products"):

            gr.Markdown("<h1>🔎 Search Stored Products</h1>")

            product_query_input = gr.Textbox(
                label="Search by Description",
                placeholder="e.g. smart assistant for homes"
            )

            search_btn = gr.Button("🔍 Search")

            search_results_output = gr.Dataframe(
                headers=["Product Name", "Description", "Image Prompt"],
                datatype=["str", "str", "str"],
                label="Matching Products",
                interactive=False,
                wrap=True
            )

            search_btn.click(
                fn=search_similar_products,
                inputs=product_query_input,
                outputs=search_results_output
            )

    gr.Markdown("<center><small style='color:white;'>Built with ❤️ using Gradio | Styled for inspiration ✨</small></center>")

demo.launch()


It looks like you are running Gradio on a hosted a Jupyter notebook. For the Gradio app to work, sharing must be enabled. Automatically setting `share=True` (you can turn this off by setting `share=False` in `launch()` explicitly).

Colab notebook detected. To show errors in colab notebook, set debug=True in launch()
* Running on public URL: https://f492c3b1b1402eb327.gradio.live

This share link expires in 1 week. For free permanent hosting and GPU upgrades, run `gradio deploy` from the terminal in the working directory to deploy to Hugging Face Spaces (https://huggingface.co/spaces)
